## CAGE raw energy spectrum checker

This notebook is intended to complement `energy_cal.py`.  
We use the interactive mode to load a raw spectrum from a particular set of cycle files, and use it to pick out the raw locations of the peaks, which can then be added to `metadata/input_peaks.json` as input guesses.

Run this notebook using the `legend-base` Shifter image.  [Here are the instructions to set this up.](https://github.com/legend-exp/legend/wiki/Computing-Resources-at-NERSC)

In [ ]:
# install user prerequisites
# !pip install ipympl --user

# Use this at NERSC to get interactive plots.
%matplotlib widget

import os, h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pygama import DataGroup, lh5
import pygama.analysis.histograms as pgh

#### >>> Users, set config here ! <<<
Set the query here to DataGroup to load files.  You may want to refer to `runDB.json` to see how to vary this.  Here we also set the energy parameter of interest. 

In [ ]:
# fileDB query
que = 'run==180 and cycle > 1651'
# que = 'run==66 and cycle > 885'

# energy estimator of interest
etype = 'energy'

# lh5 table name
tb_in = 'ORSIS3302DecoderForEnergy/dsp'

# uncalibrated energy range
xlo, xhi, xpb = 0, 10000, 10  # good for trapEmax and trapEftp
xlo, xhi, xpb = 0, 3e6, 10000

# load the fileDB and make sure the entries exist
dg = DataGroup('cage.json', load=True)
dg.fileDB.query(que, inplace=True)
if len(dg.fileDB)==0:
    print('Error, no files found.  Check your query, and fileDB.h5.')

ecal_cols = ['run', 'cycle', 'skip', 'runtype', 'startTime', 'threshold', 'stopTime', 'runtime']
dg.fileDB[ecal_cols]

#### Load data
Here we use DataGroup's fileDB to select files, retrieve DSP data,
and show some information about what we've selected.

In [ ]:
# essentially the same code as in energy_cal::check_raw_spectrum

# load numpy arrays of uncalibrated energy
dsp_list = dg.lh5_dir + dg.fileDB['dsp_path'] + '/' + dg.fileDB['dsp_file']
raw_data = lh5.load_nda(dsp_list, [etype], tb_in, verbose=False)

# get runtime
runtime_min = dg.fileDB['runtime'].sum()

# print columns of table
with h5py.File(dsp_list.iloc[0], 'r') as hf:
    print('\nLH5 columns:', list(hf[f'{tb_in}'].keys()))
    
# histogram energy data for this estimator and normalize by runtime
data = raw_data[etype]
hist, bins, var = pgh.get_hist(data, range=(xlo, xhi), dx=xpb)
bins = bins[1:] # trim zero bin, not needed with ds='steps'
hist_rt = np.divide(hist, runtime_min * 60)

print(f'\nRaw E: {etype}, {len(data)} cts, runtime: {runtime_min:.2f} min')

#### Create interactive spectrum

In [ ]:
%matplotlib widget
plt.semilogy(bins, hist_rt, ds='steps', c='b', lw=1, label=etype)
plt.xlabel(etype, ha='right', x=1)
plt.ylabel(f'cts/sec, {xpb}/bin', ha='right', y=1)
plt.show()